In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
import pandas as pd
import torch
import numpy as np
import random
import re
import spacy
import shutil
import matplotlib.pyplot as plt
!pip install transformers
!pip install optuna
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, BertTokenizer, AutoModel, BertModel
from torch.utils.data import Dataset, TensorDataset, DataLoader, random_split, RandomSampler, SequentialSampler
import time
import optuna
from scipy.stats import spearmanr
import psycopg2
import csv
import math
import os
from sqlalchemy import create_engine
from optuna.storages import RDBStorage
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 109.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 12.9 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
path_base = '/content/drive/MyDrive/Tesis/SICK/transformer_tunned_BERT/uncase_base/study_optuna/'
file_root = '/content/drive/MyDrive/Tesis/SICK/uncase_based/SICK.txt'
with open(file_root, 'r') as temp_f:
    # get No of columns in each line
    col_count = [ len(l.split("\t")) for l in temp_f.readlines() ]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def getSICKSents(filename='sts-train.csv', set_type='TRAIN', encoding='utf-8'):
  f = open(filename, 'r', encoding=encoding)
  s1, s2, target = [], [], []
  for i, line in enumerate(f):
    example = re.split(r'\t+', line)
    if i >= 1:
      if example[-1].strip() == set_type:
        s2.append(example[1])
        s1.append(example[2])
        target.append(float(example[4]))
  print("samples: {}".format(len(target)))
  return s1, s2, target

In [ ]:
s1_train,s2_train,target_train = getSICKSents(file_root, 'TRAIN')

samples: 4439


In [ ]:
s1_test,s2_test,target_test= getSICKSents(file_root, 'TEST')

samples: 4906


In [ ]:
s1_dev,s2_dev,target_dev= getSICKSents(file_root, 'TRIAL')

samples: 495


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

Device: cuda


In [ ]:
BATCH_SIZE = 32
MAX_LEN = 128
CORPUS = 'SICK'
BERT_PATH = "bert-base-uncased"

In [ ]:
TOKENIZER = BertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True)

(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

(…)bert-base-uncased/resolve/main/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)base-uncased/resolve/main/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)rt-base-uncased/resolve/main/config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def encode_sents(sents1, sents2):
    input_ids_ = []
    attention_masks_ = []
    type_ids_ = []
    for i, sent1 in enumerate(sents1):
        encoded_dict = TOKENIZER.encode_plus(
                            sent1,                      # Sentence 1 to encode.
                            sents2[i],                  # Sentence 2 to encode.
                            add_special_tokens = True,  # Add '[CLS]' and '[SEP]'
                            truncation = True,
                            max_length = MAX_LEN,       # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                      )

        # Add the encoded sentence to the list.
        input_ids_.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks_.append(encoded_dict['attention_mask'])

        type_ids_.append(encoded_dict['token_type_ids'])
    return input_ids_, attention_masks_, type_ids_

In [ ]:
def getEncodedTensors(s1, s2, labels):
    input_ids, attention_masks, type_ids = encode_sents(s1, s2)
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    type_ids = torch.cat(type_ids, dim=0)
    labels = torch.tensor(labels)
    return input_ids, attention_masks, type_ids, labels

In [ ]:
input_ids_train,attention_masks_train,type_ids_train,labels_train = getEncodedTensors(s1_train,s2_train,target_train)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
input_ids_eval, attention_masks_eval, type_ids_eval, labels_eval = getEncodedTensors(s1_dev, s2_dev, target_dev)

In [ ]:
# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(input_ids_train, attention_masks_train, type_ids_train, labels_train)
val_dataset = TensorDataset(input_ids_eval, attention_masks_eval, type_ids_eval, labels_eval)

In [ ]:
# Create the DataLoaders for our training and test sets.
# We'll take training samples in random order.
torch.manual_seed(42)
train_loader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = BATCH_SIZE # Trains with this batch size.
            #num_workers=2
        )
torch.manual_seed(42)
val_loader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = BATCH_SIZE # Evaluate with this batch size.
            #num_workers=2
        )

In [ ]:
# See first batch
#batch = next(iter(train_loader))
#print(batch[0]) # 0 -> input_ids , 1 -> attention_masks, 2 -> type_ids, 3 -> targets
#print(train_loader.batch_size) #tamaño del batch
print('Number of train batches: {}'.format(len(train_loader)))# número de batches
print('Number of evaluation batches: {}'.format(len(val_loader)))

Number of train batches: 139
Number of evaluation batches: 16


In [ ]:
class BertBaseUncasedBiLSTMRegressor(nn.Module):
  def __init__(self, output_dim=1, attentions=True, hidden_dim=768, dropout_output=0.1, dropout_lstm=0.1, n_layers = 1, bidirectional=True, freeze=True, batch_first=True):
      super().__init__()
      self.bert = BertModel.from_pretrained(BERT_PATH, output_attentions=attentions) #load the model
      for param in self.bert.parameters():
          param.requires_grad = not freeze #True -> variable, False -> fixed
      self.bilstm = nn.LSTM(input_size=self.bert.config.hidden_size, hidden_size=hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout_lstm, batch_first=batch_first)
      self.regressor = nn.Linear(hidden_dim * 2, output_dim)
      self.dropout = nn.Dropout(dropout_output)


  def forward(self, input_ids, type_ids, mask):

      bert_output = self.bert(input_ids=input_ids, token_type_ids = type_ids, attention_mask= mask)
      attention_mask = mask.unsqueeze(-1).expand(bert_output.last_hidden_state.shape).float()
      masked_last_hidden_state = bert_output.last_hidden_state * attention_mask
      """
      The bilstm_output is the tensor of all the hidden state from each time step in the RNN(LSTM), and the hidden state (h_n) returned
      by the RNN(LSTM) is the last hidden state from the last time step from the input sequence
      """
      bilstm_output, (h_n, _) = self.bilstm(masked_last_hidden_state)
      #h_n = [num layers * num directions, batch size, hid dim]
      # obtain the final hidden states from both directions of the BiLSTM
      bilstm_output = self.dropout(bilstm_output)

      # obtain the final hidden states from both directions of the BiLSTM
      final_hidden_state = torch.cat((bilstm_output[:, -1, :self.bilstm.hidden_size],
                                      bilstm_output[:, 0, self.bilstm.hidden_size:]), dim=1)

      logits = self.regressor(final_hidden_state)
      return logits, bert_output

In [ ]:
def pearson_corr(y_pred, y_true):
    all_preds = np.concatenate(y_pred)
    all_targets = np.concatenate(y_true)
    return np.corrcoef(all_preds, all_targets)[0, 1]

In [ ]:
def spearman_corr(y_pred, y_true):
    all_preds = np.concatenate(y_pred)
    all_targets = np.concatenate(y_true)

    corr, _ = spearmanr(all_preds, all_targets)
    return corr

In [ ]:
def train_loop(model,loader,optimizer,criterion,device,clip = 1.0):
    #Training loop
    model.train()
    loss_sum = 0
    all_preds = []
    all_targets = []
    seed = 42
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    for i, batch in enumerate(loader):

        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_type_ids = batch[2].to(device)
        labels = batch[3].to(device)

        optimizer.zero_grad()
        #Forward
        outputs, _ = model(input_ids=b_input_ids, type_ids = b_type_ids, mask = b_attention_mask)
        outputs = outputs.squeeze(-1)
        #Loss
        loss = criterion(outputs.view(-1), labels.float())
        #Backprop
        loss.backward()
        #prevent gradients from exploding
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        #Update params
        optimizer.step()

        loss_sum += loss.item()

        all_preds.append(outputs.detach().cpu().numpy())
        all_targets.append(labels.float().detach().cpu().numpy())

    epoch_train_loss = loss_sum / len(loader)
    epoch_train_pearson = pearson_corr(all_preds, all_targets)
    epoch_train_spearman = spearman_corr(all_preds, all_targets)

    return epoch_train_loss, epoch_train_pearson, epoch_train_spearman


In [ ]:
def evaluation_loop(model,loader,criterion,device):
    #Evaluation loop
    seed = 42
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    model.eval()
    with torch.no_grad():
        loss_sum = 0
        all_preds = []
        all_targets = []

        for i, batch in enumerate(loader):
            b_input_ids = batch[0].to(device)
            b_attention_mask = batch[1].to(device)
            b_type_ids = batch[2].to(device)
            labels = batch[3].to(device)

            #Forward
            outputs, _ = model(input_ids=b_input_ids, type_ids = b_type_ids, mask = b_attention_mask)
            outputs = outputs.squeeze(-1)
            #Loss
            loss = criterion(outputs.view(-1), labels.float())

            loss_sum += loss.item()

            all_preds.append(outputs.detach().cpu().numpy())
            all_targets.append(labels.float().detach().cpu().numpy())

        epoch_dev_loss = loss_sum / len(loader)
        epoch_dev_pearson = pearson_corr(all_preds, all_targets)
        epoch_dev_spearman = spearman_corr(all_preds, all_targets)

    return epoch_dev_loss, epoch_dev_pearson, epoch_dev_spearman

In [ ]:
def save_metrics_dataframe(metrics_dict, study_name, path_base):
  # Abrir archivo en modo escritura y especificar el separador de campos
  path = path_base + 'metrics_' + study_name + ".csv"
  with open(path, "a+", newline="") as f:
      metrics = [metrics_dict]
      # Crear objeto escritor CSV y especificar el separador de campos
      metrics_csv = csv.DictWriter(f, fieldnames=['train_loss','valid_loss','spearman_train','spearman_val','pearson_train','pearson_val'],delimiter=",")

      # Verificar si se ha escrito el encabezado del archivo
      if f.tell() == 0:
        metrics_csv.writeheader()

      # Escribir cada fila de datos
      for metric in metrics:
          metrics_csv.writerow(metric)

In [ ]:
study_name = 'study-bert-bilstm-output-base-sick'

In [ ]:
def isNan(value):
  return 0 if math.isnan(value) else value

In [ ]:
def objective(trial):
  params = {
      "num_layers": trial.suggest_int("num_layer", 1, 3),
      "dropout_output": trial.suggest_uniform("dropout_output", 0, 0.8),
      "dropout_lstm": trial.suggest_uniform("dropout_lstm", 0, 0.8),
      "freeze" : trial.suggest_categorical('freeze', [True, False]),
      "optimizer": trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
      "learning_rate": trial.suggest_loguniform("learning_rate",1e-5, 1e-4)
  }

  model = BertBaseUncasedBiLSTMRegressor(dropout_output=params["dropout_output"],dropout_lstm=params["dropout_lstm"],freeze=params["freeze"],n_layers = params["num_layers"]).to(device)
  criterion = nn.MSELoss()
  optimizer = getattr(optim, params["optimizer"])(model.parameters(), lr=params["learning_rate"])

  NUM_EPOCHS = 5
  best_valid_loss = float('inf')
  MODEL_FILE_NAME = CORPUS+'_'+BERT_PATH+'_'+str(MAX_LEN)+'_tunned_model.pt'
  history = {"train": {"loss": []}, "test": {"loss": []}}

  torch.cuda.empty_cache()
  seed = 42
  np.random.seed(seed)
  random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)

  train_loss_list, val_loss_list, spearman_train, spearman_val, pearson_train, pearson_val = [], [], [], [], [], []

  for epoch in range(NUM_EPOCHS):

    start_time = time.time()

    epoch_train_loss, epoch_train_pearson, epoch_train_spearman = train_loop(model,train_loader,optimizer,criterion,device)
    epoch_dev_loss, epoch_dev_pearson, epoch_dev_spearman = evaluation_loop(model,val_loader,criterion,device)

    elapsed_time = time.time() - start_time

    if trial.should_prune():
      raise optuna.exceptions.TrialPruned()

    #nos quedamos con el modelo que tiene mejor pérdida de validación
    if epoch_dev_loss < best_valid_loss:
      best_valid_loss = epoch_dev_loss
      torch.save(model.state_dict(), MODEL_FILE_NAME)

    train_loss_list.append(isNan(epoch_train_loss))
    val_loss_list.append(isNan(epoch_dev_loss))
    spearman_train.append(isNan(epoch_train_spearman))
    spearman_val.append(isNan(epoch_dev_spearman))
    pearson_train.append(isNan(epoch_train_pearson))
    pearson_val.append(isNan(epoch_dev_pearson))

    history["train"]["loss"].append(epoch_train_loss)
    history["test"]["loss"].append(epoch_dev_loss)

    print('-' * 80)
    print(f'Epoch: {epoch+1:03}/{NUM_EPOCHS} | Time: {elapsed_time:.4f}s | Train loss: {epoch_train_loss:.4f} | Dev loss: {epoch_dev_loss:.4f}')
    print(f'Train Pearson Coef: {epoch_train_pearson:.4f} | Dev Pearson Coef: {epoch_dev_pearson:.4f}')
    print(f'Train Spearman Coef: {epoch_train_spearman:.4f} | Dev Spearman Coef: {epoch_dev_spearman:.4f}')

  metrics_dict = {
      'train_loss' : train_loss_list,
      'valid_loss' : val_loss_list,
      'spearman_train': spearman_train,
      'spearman_val': spearman_val,
      'pearson_train': pearson_train,
      'pearson_val': pearson_val
  }

  save_metrics_dataframe(metrics_dict, study_name, path_base)

  return best_valid_loss

In [ ]:
storage_path = path_base + "optuna_study_bilstm_output_SICK_.db"

In [ ]:
# Define el estudio de Optuna utilizando el storage
study = optuna.create_study(study_name=study_name, direction="minimize", storage=f"sqlite:///{storage_path}", load_if_exists=True)

[I 2023-11-17 04:17:08,744] A new study created in RDB with name: study-bert-bilstm-output-base-sick


In [ ]:
study.optimize(objective, n_trials=15)

<ipython-input-31-b4102fbb518e>:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "dropout_output": trial.suggest_uniform("dropout_output", 0, 0.8),
<ipython-input-31-b4102fbb518e>:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "dropout_lstm": trial.suggest_uniform("dropout_lstm", 0, 0.8),
<ipython-input-31-b4102fbb518e>:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate",1e-5, 1e-4)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

--------------------------------------------------------------------------------
Epoch: 001/5 | Time: 60.9189s | Train loss: 1.7793 | Dev loss: 1.0892
Train Pearson Coef: 0.1382 | Dev Pearson Coef: 0.6470
Train Spearman Coef: 0.2623 | Dev Spearman Coef: 0.6174
--------------------------------------------------------------------------------
Epoch: 002/5 | Time: 63.4619s | Train loss: 0.6074 | Dev loss: 0.9898
Train Pearson Coef: 0.6503 | Dev Pearson Coef: 0.7235
Train Spearman Coef: 0.5945 | Dev Spearman Coef: 0.6795
--------------------------------------------------------------------------------
Epoch: 003/5 | Time: 63.0831s | Train loss: 0.5017 | Dev loss: 0.8895
Train Pearson Coef: 0.7235 | Dev Pearson Coef: 0.7530
Train Spearman Coef: 0.6555 | Dev Spearman Coef: 0.7055
--------------------------------------------------------------------------------
Epoch: 004/5 | Time: 63.0584s | Train loss: 0.4373 | Dev loss: 0.8124
Train Pearson Coef: 0.7645 | Dev Pearson Coef: 0.7715
Train Spearm

[I 2023-11-17 04:22:58,427] Trial 0 finished with value: 0.780758436769247 and parameters: {'num_layer': 2, 'dropout_output': 0.5431572854028318, 'dropout_lstm': 0.7387679569266918, 'freeze': True, 'optimizer': 'RMSprop', 'learning_rate': 1.074781853666988e-05}. Best is trial 0 with value: 0.780758436769247.


--------------------------------------------------------------------------------
Epoch: 005/5 | Time: 63.2334s | Train loss: 0.3890 | Dev loss: 0.7808
Train Pearson Coef: 0.7940 | Dev Pearson Coef: 0.7827
Train Spearman Coef: 0.7171 | Dev Spearman Coef: 0.7379


...

--------------------------------------------------------------------------------
Epoch: 005/5 | Time: 137.6865s | Train loss: 0.1199 | Dev loss: 0.4141
Train Pearson Coef: 0.9420 | Dev Pearson Coef: 0.8607
Train Spearman Coef: 0.9190 | Dev Spearman Coef: 0.8090


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.6737725688481486 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


--------------------------------------------------------------------------------
Epoch: 001/5 | Time: 100.0243s | Train loss: 1.4989 | Dev loss: 0.4139
Train Pearson Coef: 0.3774 | Dev Pearson Coef: 0.8461
Train Spearman Coef: 0.4174 | Dev Spearman Coef: 0.7989
--------------------------------------------------------------------------------
Epoch: 002/5 | Time: 100.0643s | Train loss: 0.2629 | Dev loss: 0.2690
Train Pearson Coef: 0.8668 | Dev Pearson Coef: 0.8608
Train Spearman Coef: 0.7949 | Dev Spearman Coef: 0.8120
--------------------------------------------------------------------------------
Epoch: 003/5 | Time: 100.1080s | Train loss: 0.1661 | Dev loss: 0.3094
Train Pearson Coef: 0.9180 | Dev Pearson Coef: 0.8608
Train Spearman Coef: 0.8741 | Dev Spearman Coef: 0.8071
--------------------------------------------------------------------------------
Epoch: 004/5 | Time: 100.0970s | Train loss: 0.1168 | Dev loss: 0.2502
Train Pearson Coef: 0.9433 | Dev Pearson Coef: 0.8714
Train Sp

[I 2023-11-17 06:27:56,959] Trial 14 finished with value: 0.25021771527826786 and parameters: {'num_layer': 1, 'dropout_output': 0.3245200551166244, 'dropout_lstm': 0.6737725688481486, 'freeze': False, 'optimizer': 'Adam', 'learning_rate': 2.4959606327728666e-05}. Best is trial 14 with value: 0.25021771527826786.


--------------------------------------------------------------------------------
Epoch: 005/5 | Time: 100.1456s | Train loss: 0.0862 | Dev loss: 0.3371
Train Pearson Coef: 0.9586 | Dev Pearson Coef: 0.8694
Train Spearman Coef: 0.9394 | Dev Spearman Coef: 0.8166


In [ ]:
df_optuna = study.trials_dataframe(attrs=("number", "value", "params", "state")).dropna(subset=['value']) #eliminar valores NaN
# Leer el archivo metrics.CSV y crear un DataFrame
path_metrics = path_base + 'metrics_' + study_name + ".csv"
df_metrics = pd.read_csv(path_metrics).dropna(subset=['train_loss'])
print(len(df_optuna))
print(len(df_metrics ))

In [ ]:
#df_metrics = df_metrics[['train_loss', 'valid_loss', 'pearson_train', 'pearson_val', 'spearman_train', 'spearman_val']]

In [ ]:
#df_metrics = df_metrics.rename(columns={'train_loss': 'train_loss', 'valid_loss': 'val_loss', 'pearson_train': 'Pearson_train','pearson_val': 'Pearson_val', 'spearman_train': 'Spearman_train', 'spearman_val': 'Spearman_val'})

In [ ]:
df_metrics

In [ ]:
df_old = pd.DataFrame()

In [ ]:
df_old = pd.read_csv(path_base + study_name + "_COMPLETE_SORT.csv")
print("Old trials: ", len(df_old))

In [ ]:
# Seleccionar los valores de df_optuna que no están ya en df_old
if len(df_optuna)-len(df_metrics) == len(df_old):
    df_optuna_filter = df_optuna.tail(len(df_metrics)).reset_index(drop=True)
else:
    # Seleccionar los valores de df_optuna que no están ya en df_old
    mask = ~df_optuna['value'].isin(df_old['value'])
    df_optuna_filter = df_optuna[mask].reset_index(drop=True)
    print(len(df_optuna_filter))

In [ ]:
def concat(df_optuna_filter, df_metrics, df_old):
  # Unir dataframe de metrics con dataframe de optuna de forma horizontal y manejar los valores inexistentes
  df_join = pd.concat([df_optuna_filter, df_metrics], axis=1, sort=False)
  # Unir dataframe de metrics con dataframe de optuna de forma vertical y manejar los valores inexistentes
  df_complete_updated = pd.concat([df_old, df_join], axis=0, sort=False).drop_duplicates()
  print("All trials: ", len(df_complete_updated))
  assert len(df_join) + len(df_old) == len(df_complete_updated)
  return df_complete_updated

In [ ]:
if len(df_optuna_filter) == len(df_metrics):
  df_complete_updated = concat(df_optuna_filter, df_metrics, df_old).dropna(subset=['train_loss', 'value'])

if len(df_optuna_filter) > len(df_metrics):
  num_rows_to_add = len(df_optuna_filter) - len(df_metrics)
  new_rows = [[np.nan] * len(df_metrics.columns) for i in range(num_rows_to_add)]
  new_df = pd.DataFrame(new_rows, columns=df_metrics.columns)
  df_metrics = pd.concat([new_df, df_metrics], ignore_index=True)
  df_complete_updated = concat(df_optuna_filter, df_metrics, df_old).dropna(subset=['train_loss','value'])

In [ ]:
if len(df_optuna) == len(df_metrics):
  df_complete_updated = pd.concat([df_optuna, df_metrics], axis=1, sort=False)

In [ ]:
df_complete_updated

In [ ]:
# Ordenar el DataFrame unido por valor
df = df_complete_updated.sort_values(by='value', ascending=True)
# Obtener la prueba con el mejor valor
best_trial = df.iloc[0]
print("Best trial: ")
print(best_trial)

Best trial: 
number                                                                  14
value                                                             0.250218
params_dropout_lstm                                               0.673773
params_dropout_output                                              0.32452
params_freeze                                                        False
params_learning_rate                                              0.000025
params_num_layer                                                         1
params_optimizer                                                      Adam
state                                                             COMPLETE
train_loss               [1.4989120154072055, 0.2628681266800963, 0.166...
valid_loss               [0.4139465820044279, 0.26903006434440613, 0.30...
spearman_train           [0.41744902318206883, 0.7948516397133611, 0.87...
spearman_val             [0.7989038598196733, 0.8120247280141295, 0.807...
pearson_trai

In [ ]:
#Guardar CSV completo y ordenado
# ruta de acceso a la carpeta de Google Drive montada
#os.remove(path_base + study_name + "_COMPLETE_SORT.csv")
path_ = path_base + study_name + "_COMPLETE_SORT.csv"
# guardar el archivo CSV en Google Drive
df.to_csv(path_, index=False)
files.download(path_)
#os.remove(path_metrics)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>